<a href="https://colab.research.google.com/github/jlonge4/gen_ai_utils/blob/main/qdrant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -qU langchain openai faiss-cpu sentence_transformers qdrant-client==1.4.0 --quiet

In [ ]:
# from langchain import OpenAI
from langchain.chat_models import ChatOpenAI
import os
import openai

key = 'key'
llm = ChatOpenAI(
    openai_api_key=key,
    temperature=0,
    model_name="gpt-3.5-turbo"
)

In [ ]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


In [ ]:
# from sentence_transformers import SentenceTransformer
# # initialize sentence transformer model
# model = SentenceTransformer('bert-base-nli-mean-tokens')
# # create sentence embeddings


In [ ]:
from langchain.schema import Document
from langchain.document_loaders import WebBaseLoader

two_web_links = ["https://www.databricks.com/","https://help.databricks.com","https://databricks.com/try-databricks","https://help.databricks.com/s/","https://docs.databricks.com","https://kb.databricks.com/","http://docs.databricks.com/getting-started/index.html","http://docs.databricks.com/introduction/index.html","http://docs.databricks.com/getting-started/tutorials/index.html","http://docs.databricks.com/release-notes/index.html","http://docs.databricks.com/ingestion/index.html","http://docs.databricks.com/exploratory-data-analysis/index.html","http://docs.databricks.com/data-preparation/index.html","http://docs.databricks.com/data-sharing/index.html","http://docs.databricks.com/marketplace/index.html","http://docs.databricks.com/workspace-index.html","http://docs.databricks.com/machine-learning/index.html","http://docs.databricks.com/sql/index.html","http://docs.databricks.com/delta/index.html","http://docs.databricks.com/dev-tools/index.html","http://docs.databricks.com/integrations/index.html","http://docs.databricks.com/administration-guide/index.html","http://docs.databricks.com/security/index.html","http://docs.databricks.com/data-governance/index.html","http://docs.databricks.com/lakehouse-architecture/index.html","http://docs.databricks.com/reference/api.html","http://docs.databricks.com/resources/index.html","http://docs.databricks.com/whats-coming.html","http://docs.databricks.com/archive/index.html","http://docs.databricks.com/lakehouse/index.html","http://docs.databricks.com/getting-started/quick-start.html","http://docs.databricks.com/getting-started/etl-quick-start.html","http://docs.databricks.com/getting-started/lakehouse-e2e.html","http://docs.databricks.com/getting-started/free-training.html","http://docs.databricks.com/sql/language-manual/index.html","http://docs.databricks.com/error-messages/index.html","http://www.apache.org/","https://databricks.com/privacy-policy","https://databricks.com/terms-of-use"]
web_links = ["https://www.pinecone.io/learn/vector-embeddings/", "https://www.featureform.com/post/the-definitive-guide-to-embeddings"]
loader = WebBaseLoader(two_web_links)
documents = loader.load()

In [ ]:
from importlib.metadata import metadata
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

In [ ]:
docs = []
for d in all_splits:
  docs.append(d.page_content)

In [ ]:
embeddings = model.encode(docs)


In [ ]:
embeddings.shape

(522, 768)

In [ ]:
from langchain.vectorstores import Qdrant

In [ ]:
# langchain implementation
qdrant = Qdrant.from_documents(
    all_splits,
    embedding_function,
    path="local_qd",
    collection_name="my_documents",
)

In [ ]:
# # re-initialize index for use from disk
# from qdrant_client import QdrantClient

# client = QdrantClient(path='test_qd')
# # qdrant = Qdrant(client, collection_name="my_documents", embedding_function=model)

In [ ]:
# # Prepare your documents, metadata, and IDs
# docs = all_splits
# # metadatas = [
# #     {"source": "Langchain-docs"},
# #     {"source": "Linkedin-docs"},
# # ]
# # ids = [42, 2]

# # Use the new add method
# client.add(collection_name="test_qd", docs={"documents": docs, "metadatas": [], "ids": 0})

# search_result = client.query(collection_name="test_qd", query_texts=["This is a query document"])
# print(search_result)

In [ ]:
question = 'What is databricks used for?'

In [ ]:
from qdrant_client.http import models
filter = models.Filter(should=[models.FieldCondition(key='metadata.source',
                                          match=models.MatchValue(value="http://docs.databricks.com/introduction/index.html"))])

In [ ]:
%%time
q = qdrant.similarity_search_with_relevance_scores(question, filter=filter)

CPU times: user 14.9 ms, sys: 6.51 ms, total: 21.4 ms
Wall time: 17.1 ms


In [ ]:
%%time
query = qdrant.similarity_search_with_relevance_scores(question)

CPU times: user 11.2 ms, sys: 4.15 ms, total: 15.4 ms
Wall time: 12 ms


In [ ]:
from pprint import pprint
pprint('Without Filtering: ')
pprint(query[0][0].metadata['source'])
pprint(query[0])
pprint('\n')
pprint('Filtered: ')
pprint(q[0][0].metadata['source'])
pprint(q[0])

'Without Filtering: '
'http://docs.databricks.com/getting-started/quick-start.html'
(Document(page_content='What is Databricks?\nNavigate the workspace\nIntroduction to Databricks notebooks and Visualizations in Databricks notebooks\nLibraries\nClusters and Introduction to Databricks Workflows\nLoad data using the add data UI and Create or modify a table using file upload\nDiscover and manage data using Data Explorer\nDeveloper tools and guidance\nTechnology partners\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n          © Databricks 2023. All rights reserved. Apache, Apache Spark, Spark, and the Spark logo are trademarks of the Apache Software Foundation.\n      \n\nSend us feedback\n        \n     | Privacy Policy | Terms of Use', metadata={'source': 'http://docs.databricks.com/getting-started/quick-start.html', 'title': 'Tutorial: Query data with notebooks | Databricks on AWS', 'description': 'Learn data science basics on Databricks. Create a cluster, run a notebook, create 

In [ ]:
# ask question before relevant docs are added
Q2 = "How are embeddings created?"
qdrant.similarity_search_with_relevance_scores(Q2, k=2)


[(Document(page_content='External locations can be defined as an entire storage container, but often point to a directory nested in a container.\nThe correct format for an external location path is "s3://bucket-name/path/to/external_location".', metadata={'source': 'http://docs.databricks.com/getting-started/lakehouse-e2e.html', 'title': 'Tutorial: Run an end-to-end lakehouse analytics pipeline | Databricks on AWS', 'description': 'Run your first end-to-end analytics pipeline in the Databricks Lakehouse.', 'language': 'en-US'}),
  0.17959470297373423),
 (Document(page_content='be an infringer of our or any third party’s intellectual property rights of any kind whatsoever.You agree that you will not, and will not allow or authorize any third party to, post Submissions containing:Anything that is or may be (a) threatening, harassing, degrading, abusive or hateful; (b) an incitement to violence, terrorism or other wrongdoing; (c) defamatory or libelous; (d) invasive of privacy rights; (e)

In [ ]:
qdrant.client.count(
    collection_name="my_documents",
    exact=True
)

CountResult(count=522)

In [ ]:
from langchain.schema import Document
from langchain.document_loaders import WebBaseLoader

two_web_links = ["https://www.databricks.com/","https://help.databricks.com","https://databricks.com/try-databricks","https://help.databricks.com/s/","https://docs.databricks.com","https://kb.databricks.com/","http://docs.databricks.com/getting-started/index.html","http://docs.databricks.com/introduction/index.html","http://docs.databricks.com/getting-started/tutorials/index.html","http://docs.databricks.com/release-notes/index.html","http://docs.databricks.com/ingestion/index.html","http://docs.databricks.com/exploratory-data-analysis/index.html","http://docs.databricks.com/data-preparation/index.html","http://docs.databricks.com/data-sharing/index.html","http://docs.databricks.com/marketplace/index.html","http://docs.databricks.com/workspace-index.html","http://docs.databricks.com/machine-learning/index.html","http://docs.databricks.com/sql/index.html","http://docs.databricks.com/delta/index.html","http://docs.databricks.com/dev-tools/index.html","http://docs.databricks.com/integrations/index.html","http://docs.databricks.com/administration-guide/index.html","http://docs.databricks.com/security/index.html","http://docs.databricks.com/data-governance/index.html","http://docs.databricks.com/lakehouse-architecture/index.html","http://docs.databricks.com/reference/api.html","http://docs.databricks.com/resources/index.html","http://docs.databricks.com/whats-coming.html","http://docs.databricks.com/archive/index.html","http://docs.databricks.com/lakehouse/index.html","http://docs.databricks.com/getting-started/quick-start.html","http://docs.databricks.com/getting-started/etl-quick-start.html","http://docs.databricks.com/getting-started/lakehouse-e2e.html","http://docs.databricks.com/getting-started/free-training.html","http://docs.databricks.com/sql/language-manual/index.html","http://docs.databricks.com/error-messages/index.html","http://www.apache.org/","https://databricks.com/privacy-policy","https://databricks.com/terms-of-use"]
web_links = ["https://www.pinecone.io/learn/vector-embeddings/", "https://www.featureform.com/post/the-definitive-guide-to-embeddings"]
loader = WebBaseLoader(web_links)
documents = loader.load()

In [ ]:
from importlib.metadata import metadata
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

In [ ]:
metadatas = []
for d in all_splits:
  #get just the metadata
  metadatas.append(d.metadata)

In [ ]:
texts = []
for d in all_splits:
  # get just the text
  texts.append(d.page_content)

In [ ]:
metadatas[1]

{'source': 'https://www.pinecone.io/learn/vector-embeddings/',
 'title': 'What are Vector Embeddings | Pinecone',
 'description': 'Vector embeddings are one of the most fascinating and useful concepts in machine learning. They are central to many NLP, recommendation, and search algorithms. If you’ve ever used things like recommendation engines, voice assistants, language translators, you’ve come across systems that rely on embeddings.',
 'language': 'en'}

In [ ]:
# add texts and associated metadata to the collection
qdrant.add_texts(texts, metadatas=metadatas)

['947d05c5a1f54aa0ae73a12e9ff36a02',
 '4fa987a4362743be9d6bd0025bb6d36d',
 'dec8f1c8720a40c7b13a25e67a89529e',
 'b4f27b60b1764721a4bca12560d268f6',
 'c990a34590a847fab2d4adf9af44b3ff',
 '3e60580276504db7add2a09ffb8f7bc6',
 'e5c40306348c49eb9577fe1466eedd18',
 'df2da5b0522443d19f201affd9ee1c50',
 '81381cbaf8394317a5366a21631a3491',
 '34dc55e512e948968929daacd674f0cd',
 '9f6610cff90c403f9d4d48b5de964fa8',
 '9f9c1425a97e42a0bc06ecce95d928fc',
 '9a4abc84527d4682b4796d2a8d71a468',
 'cbb64da403ab4078ac59c9a35a50f425',
 '723c07f3a530427a869c3069738d35fd',
 '8979443ea46e448fb0a031e67aaec3a8',
 '423703d653e646b7aa93dd24534db3b1',
 'db425af8c7a144e6ac57e6009836d696',
 '84278a0104854843bfa4af9f50e08b24',
 '10a7efdc004a409fa3429115b57db435',
 '58c48ef8c2ce459fadfc910c6986931b',
 'e8ef121431a44dcc8df14733b3ed1930',
 '3b3c755c55ce42f6ae2a9efb607c759b',
 'a91b4a5681fa45fbb622dde648b19919',
 'f2524992a3bc46e9af3af167c0c425f1',
 '3fd9b6f5bf2a4eebb1d32cfd2409cb7b',
 '6e37daa16bfe48ed9c8caeacd2576cd0',
 

In [ ]:
qdrant.client.count(
    collection_name="my_documents",
    exact=True
)

CountResult(count=554)

In [ ]:
# ask question after relevant docs are added
qdrant.similarity_search_with_relevance_scores(Q2, k=5)


[(Document(page_content='are Embeddings created?A common way to create an embedding requires us to first set up a supervised machine learning problem. As a side-effect, training that model encodes categories into embedding vectors. For example, we can set up a model that predicts the next movie a user will watch based on what they are watching now. An embedding model will factorize the input into a vector and that vector will be used to predict the next movie. This means that similar vectors are movies that are commonly watched after similar movies. This makes for a great representation to be used for personalization. So even though we are solving a supervised problem, often called the surrogate problem, the actual creation of embeddings is an unsupervised process.Defining a surrogate problem is an art, and dramatically affects the behavior of the embeddings. For example, YouTube‚Äôs recommender team realized that using the ‚Äúpredict the next video a user is going to click on‚Äù resul

In [ ]:
qdrant.client.retrieve(
    collection_name="my_documents",
    ids=['3b3c755c55ce42f6ae2a9efb607c759b'],
)

[Record(id='3b3c755c55ce42f6ae2a9efb607c759b', payload={'page_content': 'embedding and find items that are most relevant to them. In a search engine, we can find a document that‚Äôs most similar to a search query. Nearest neighbor is a computationally expensive operation however. Performed naively it is O(N*K), where N is the number of items and K is the size of each embedding. However, in most cases when we need nearest neighbors, an approximation would suffice. If we recommend five items to a user, and one is technically the sixth closest item, the user probably won‚Äôt care. Approximate nearest neighbor (ANN) algorithms typically drop the complexity of a lookup to O(log(n)).Implementations of ANNThere are many different algorithms to efficiently find approximate nearest neighbors, and many implementations of each of them. We‚Äôll list a few of the most common algorithms and implementations below and break down how they work at a high level.Spotify‚Äôs AnnoyIn Spotify‚Äôs ANN impleme

In [ ]:
query_vec = embedding_function.embed_query('what embeddings are, how they work, and how they are often operationalized')

In [ ]:
qdrant.client.search_groups(
    collection_name="my_documents",

    # Same as in the regular search() API
    query_vector=query_vec,

    # Grouping parameters
    group_by="source", # Path of the field to group by
    limit=2,                # Max amount of groups
    group_size=2,           # Max amount of points per group

    # Lookup parameters
    with_lookup=models.WithLookup(
        # Name of the collection to look up points in
        collection="my_documents",

        # Options for specifying what to bring from the payload
        # of the looked up point, True by default
        # with_payload=["title", "text"],

        # Options for specifying what to bring from the vector(s)
        # of the looked up point, True by default
        with_vectors=False,
    )
)

GroupsResult(groups=[])

In [ ]:
ids = qdrant.client.search(
    collection_name="my_documents",
    query_filter=filter,
    query_vector=query_vec,
    limit=554)

In [ ]:
ids[1].id

'34dc55e512e948968929daacd674f0cd'

In [ ]:
id_list = []
for i in range(0, len(ids)):
  id_list.append(ids[i].id)

In [ ]:
id_list

['cbb64da403ab4078ac59c9a35a50f425',
 '34dc55e512e948968929daacd674f0cd',
 '58c48ef8c2ce459fadfc910c6986931b',
 'e8ef121431a44dcc8df14733b3ed1930',
 '81381cbaf8394317a5366a21631a3491',
 'db425af8c7a144e6ac57e6009836d696',
 '9a4abc84527d4682b4796d2a8d71a468',
 '3fd9b6f5bf2a4eebb1d32cfd2409cb7b',
 '99adc784753e41868c6ef28d16bcf848',
 '423703d653e646b7aa93dd24534db3b1',
 '10a7efdc004a409fa3429115b57db435',
 'd9e602db1e4a4b40af967217ac940da5',
 '84278a0104854843bfa4af9f50e08b24',
 'a91b4a5681fa45fbb622dde648b19919',
 '9f6610cff90c403f9d4d48b5de964fa8',
 '8979443ea46e448fb0a031e67aaec3a8',
 '3b3c755c55ce42f6ae2a9efb607c759b',
 'f2524992a3bc46e9af3af167c0c425f1',
 '723c07f3a530427a869c3069738d35fd',
 '6e37daa16bfe48ed9c8caeacd2576cd0',
 '9f9c1425a97e42a0bc06ecce95d928fc',
 'b6b8dbceeb9446b3af9437e35ee5b226',
 'ab2b4012f6534798a68c1bf7e8a36737',
 '2481f825509f4cc7b1256b54ca47ba1d']

In [ ]:
len(ids)

24